In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
cd drive/My\ Drive/Colab\ Notebooks

/content/drive/My Drive/Colab Notebooks


In [3]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [0]:
from numpy.random import seed
seed(101)
import tensorflow as tf
tf.random.set_seed(101)

import pandas as pd
import numpy as np

import tensorflow

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

import os
import cv2

import imageio
import skimage
import skimage.io
import skimage.transform

from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import itertools
import shutil
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
# Number of samples we will have in each class.
SAMPLE_SIZE = 250

# The images will all be resized to this size.
IMAGE_SIZE = 96

In [0]:
data_path = './data'
os.listdir(data_path)

['Black-grass',
 'Charlock',
 'Cleavers',
 'Common Chickweed',
 'Common wheat',
 'Fat Hen',
 'Loose Silky-bent',
 'Maize',
 'nonsegmentedv2',
 'Scentless Mayweed',
 'Shepherd_Purse',
 'Small-flowered Cranesbill',
 'Sugar beet']

In [0]:
# Create a new directory to store all available images
all_images_dir = 'all_images_dir'
os.mkdir(all_images_dir)

In [0]:
# This code copies all images from their seperate folders into the same 
# folder called all_images_dir.


folder_list = os.listdir(data_path)

for folder in folder_list:
    
    # create a path to the folder
    path = data_path + '/' + str(folder)

    # create a list of all files in the folder
    file_list = os.listdir(path)

    # move the 0 images to all_images_dir
    for fname in file_list:

        # source path to image
        src = os.path.join(path, fname)
        
        # Change the file name because many images have the same file name.
        # Add the folder name to the existing file name.
        new_fname = str(folder) + '_' + fname
        
        # destination path to image
        dst = os.path.join(all_images_dir, new_fname)
        # copy the image from the source to the destination
        shutil.copyfile(src, dst)

In [0]:
# Check how many images are in all_images_dir.
# Should be 5539.

len(os.listdir('all_images_dir'))

5539

In [13]:
# Get a list of all images in the all_images_dir folder.
image_list = os.listdir('all_images_dir')

# Create the dataframe.
df_data = pd.DataFrame(image_list, columns=['image_id'])

df_data.head()

,image_id
0,Small-flowered Cranesbill_133.png
1,Small-flowered Cranesbill_127.png
2,Small-flowered Cranesbill_125.png
3,Small-flowered Cranesbill_138.png
4,Small-flowered Cranesbill_13.png


In [14]:
# Each file name has this format:
# Loose Silky-bent_377.png

# This function will extract the class name from the file name of each image.
def extract_target(x):
    # split into a list
    a = x.split('_')
    # the target is the first index in the list
    target = a[0]
    
    return target


# create a new column called 'target'
df_data['target'] = df_data['image_id'].apply(extract_target)

df_data.head()

,image_id,target
0,Small-flowered Cranesbill_133.png,Small-flowered Cranesbill
1,Small-flowered Cranesbill_127.png,Small-flowered Cranesbill
2,Small-flowered Cranesbill_125.png,Small-flowered Cranesbill
3,Small-flowered Cranesbill_138.png,Small-flowered Cranesbill
4,Small-flowered Cranesbill_13.png,Small-flowered Cranesbill


In [15]:
df_data.shape

(5539, 2)

In [16]:
# What is the class distribution?

df_data['target'].value_counts()

Loose Silky-bent             762
Common Chickweed             713
Scentless Mayweed            607
Small-flowered Cranesbill    576
Fat Hen                      538
Sugar beet                   463
Charlock                     452
Cleavers                     335
Black-grass                  309
Shepherd                     274
Maize                        257
Common wheat                 253
Name: target, dtype: int64

In [17]:
# train_test_split

# stratify=y creates a balanced validation set.
y = df_data['target']

df_train, df_val = train_test_split(df_data, test_size=0.10, random_state=101, stratify=y)

print(df_train.shape)
print(df_val.shape)

(4985, 2)
(554, 2)


In [18]:
# Train set class distribution

df_train['target'].value_counts()

Loose Silky-bent             686
Common Chickweed             642
Scentless Mayweed            546
Small-flowered Cranesbill    518
Fat Hen                      484
Sugar beet                   417
Charlock                     407
Cleavers                     301
Black-grass                  278
Shepherd                     247
Maize                        231
Common wheat                 228
Name: target, dtype: int64

In [19]:
# Val set class distribution

df_val['target'].value_counts()

Loose Silky-bent             76
Common Chickweed             71
Scentless Mayweed            61
Small-flowered Cranesbill    58
Fat Hen                      54
Sugar beet                   46
Charlock                     45
Cleavers                     34
Black-grass                  31
Shepherd                     27
Maize                        26
Common wheat                 25
Name: target, dtype: int64

In [0]:
# folder_list = os.listdir(data_path)

classes = ['Black-grass', 'Charlock', 'Cleavers', 'Common Chickweed', 'Common wheat', 'Fat Hen', 'Loose Silky-bent', 'Maize', 
               'nonsegmentedv2', 'Scentless Mayweed', 'Shepherd_Purse', 'Small-flowered Cranesbill', 'Sugar beet']

In [0]:
# check that the folders have been created
os.mkdir('base_dir')
os.mkdir('base_dir/train_dir')
os.mkdir('base_dir/test_dir')

for x in folder_list:
    os.mkdir('base_dir/train_dir/'+x)
    os.mkdir('base_dir/test_dir/'+x)

In [0]:
# Set the id as the index in df_data
df_data.set_index('image_id', inplace=True)

In [0]:
src = 'all_images_dir/'
dst = 'base_dir/train_dir/'

for x in df_train['image_id']:
    a = x.split('_')
#     print(a[1])
    if a[1] == 'Purse':
        a[0] = a[0]+'_'+a[1]
        a[1] = a[2]
    shutil.copyfile(src+x, dst+a[0]+'/'+a[1])

In [0]:
src = 'all_images_dir/'
dst = 'base_dir/test_dir/'

for x in df_val['image_id']:
    a = x.split('_')
#     print(a[1])
    if a[1] == 'Purse':
        a[0] = a[0]+'_'+a[1]
        a[1] = a[2]
    shutil.copyfile(src+x, dst+a[0]+'/'+a[1])

In [7]:
import os, re, glob
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
  
groups_folder_path = './base_dir/train_dir/'
categories = ['Black-grass', 'Charlock', 'Cleavers', 'Common Chickweed', 'Common wheat', 'Fat Hen', 'Loose Silky-bent', 'Maize', 
                'Scentless Mayweed', 'Shepherd_Purse', 'Small-flowered Cranesbill', 'Sugar beet']
 
num_classes = len(categories)
  
image_w = 28
image_h = 28
  
X = []
Y = []
  
for idex, categorie in enumerate(categories):
    label = [0 for i in range(num_classes)]
    label[idex] = 1
    image_dir = groups_folder_path + categorie + '/'
  
    for top, dir, f in os.walk(image_dir):
        for filename in f:
            print(image_dir+filename)
            img = cv2.imread(image_dir+filename)
            img = cv2.resize(img,dsize=(IMAGE_SIZE, IMAGE_SIZE), interpolation=cv2.INTER_AREA)
            X.append(img/256)
            Y.append(label)
 
X = np.array(X)
Y = np.array(Y)
 
X_train, X_test, Y_train, Y_test = train_test_split(X,Y)
xy = (X_train, X_test, Y_train, Y_test)


./base_dir/train_dir/Black-grass/75.png
./base_dir/train_dir/Black-grass/135.png
./base_dir/train_dir/Black-grass/134.png
./base_dir/train_dir/Black-grass/32.png
./base_dir/train_dir/Black-grass/111.png
./base_dir/train_dir/Black-grass/305.png
./base_dir/train_dir/Black-grass/165.png
./base_dir/train_dir/Black-grass/290.png
./base_dir/train_dir/Black-grass/327.png
./base_dir/train_dir/Black-grass/13.png
./base_dir/train_dir/Black-grass/251.png
./base_dir/train_dir/Black-grass/183.png
./base_dir/train_dir/Black-grass/96.png
./base_dir/train_dir/Black-grass/110.png
./base_dir/train_dir/Black-grass/277.png
./base_dir/train_dir/Black-grass/166.png
./base_dir/train_dir/Black-grass/121.png
./base_dir/train_dir/Black-grass/19.png
./base_dir/train_dir/Black-grass/248.png
./base_dir/train_dir/Black-grass/279.png
./base_dir/train_dir/Black-grass/80.png
./base_dir/train_dir/Black-grass/160.png
./base_dir/train_dir/Black-grass/82.png
./base_dir/train_dir/Black-grass/295.png
./base_dir/train_dir/Bl

In [0]:
# X_train.shape
np.save("./img_data.npy", xy)

In [0]:
# End of Data Preparation
### ================================================================================== ###
# Start of Model Building

# AutoEncoder

In [30]:
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D
from keras.models import Model

# adapt this if using `channels_first` image data format
input_img = Input(shape=(IMAGE_SIZE, IMAGE_SIZE,3))

x = Conv2D(32, (3, 3), activation='relu', padding='same',kernel_initializer = 'he_uniform',name='encode_1')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same',kernel_initializer = 'he_uniform',name='encode_2')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

# at this point the representation is (7, 7, 32)

x = Conv2D(32, (3, 3), activation='relu', padding='same',kernel_initializer = 'he_uniform',name='decode_1')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same',kernel_initializer = 'he_uniform',name='decode_2')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

# this model maps an input to its reconstruction
autoencoder = Model(input_img, decoded)

# this model maps an input to its encoded representation
encoder = Model(input_img, encoded)

autoencoder.summary()

Model: "model_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        (None, 96, 96, 3)         0         
_________________________________________________________________
encode_1 (Conv2D)            (None, 96, 96, 32)        896       
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 48, 48, 32)        0         
_________________________________________________________________
encode_2 (Conv2D)            (None, 48, 48, 32)        9248      
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 24, 24, 32)        0         
_________________________________________________________________
decode_1 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
up_sampling2d_15 (UpSampling (None, 48, 48, 32)        0  

In [31]:
# compile
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

## train
from keras.callbacks import TensorBoard

history = autoencoder.fit(X_train, X_train,
                epochs=30,
                batch_size=32,
                shuffle=True,
                validation_data=(X_test, X_test))

Train on 3738 samples, validate on 1247 samples
Epoch 1/30
3738/3738 [==============================] - 19s 5ms/step - loss: 0.5853 - val_loss: 0.5708
Epoch 2/30
3738/3738 [==============================] - 4s 1ms/step - loss: 0.5658 - val_loss: 0.5615
Epoch 3/30
3738/3738 [==============================] - 4s 1ms/step - loss: 0.5606 - val_loss: 0.5621
Epoch 4/30
3738/3738 [==============================] - 4s 1ms/step - loss: 0.5579 - val_loss: 0.5595
Epoch 5/30
3738/3738 [==============================] - 4s 1ms/step - loss: 0.5567 - val_loss: 0.5599
Epoch 6/30
3738/3738 [==============================] - 4s 1ms/step - loss: 0.5564 - val_loss: 0.5577
Epoch 7/30
3738/3738 [==============================] - 4s 1ms/step - loss: 0.5558 - val_loss: 0.5570
Epoch 8/30
3738/3738 [==============================] - 4s 1ms/step - loss: 0.5554 - val_loss: 0.5565
Epoch 9/30
3738/3738 [==============================] - 4s 1ms/step - loss: 0.5552 - val_loss: 0.5596
Epoch 10/30
3738/3738 [==========

In [34]:
train_X = autoencoder.predict(X_train)
train_X.shape

(3738, 96, 96, 3)

In [46]:

kernel_size = (3,3)
pool_size= (2,2)
first_filters = 32
second_filters = 64
third_filters = 128

dropout_conv = 0.3
dropout_dense = 0.3


model = Sequential()
model.add(Conv2D(first_filters, kernel_size, activation = 'relu', 
                 input_shape = train_X.shape[1:]))
model.add(Conv2D(first_filters, kernel_size, activation = 'relu'))
model.add(Conv2D(first_filters, kernel_size, activation = 'relu'))
model.add(MaxPooling2D(pool_size = pool_size)) 
model.add(Dropout(dropout_conv))

model.add(Conv2D(second_filters, kernel_size, activation ='relu'))
model.add(Conv2D(second_filters, kernel_size, activation ='relu'))
model.add(Conv2D(second_filters, kernel_size, activation ='relu'))
model.add(MaxPooling2D(pool_size = pool_size))
model.add(Dropout(dropout_conv))

model.add(Conv2D(third_filters, kernel_size, activation ='relu'))
model.add(Conv2D(third_filters, kernel_size, activation ='relu'))
model.add(Conv2D(third_filters, kernel_size, activation ='relu'))
model.add(MaxPooling2D(pool_size = pool_size))
model.add(Dropout(dropout_conv))

model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dropout(dropout_dense))
model.add(Dense(12, activation = "softmax"))

model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 94, 94, 32)        896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 92, 92, 32)        9248      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 90, 90, 32)        9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 45, 45, 32)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 45, 45, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 43, 43, 64)        18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 41, 41, 64)       

In [0]:
model.compile(Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [60]:
X_train.shape

(3738, 96, 96, 3)

In [48]:
filepath = "model.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, 
                             save_best_only=True, mode='max')

reduce_lr = ReduceLROnPlateau(monitor='val_acc', factor=0.5, patience=3, 
                                   verbose=1, mode='max', min_lr=0.00001)
                              
                              
callbacks_list = [checkpoint, reduce_lr]
epochs = 100

model.fit(train_X, Y_train, batch_size=10,
           epochs=100,
           callbacks=callbacks_list,
           validation_split = 0.05)

Train on 3551 samples, validate on 187 samples
Epoch 1/100
3551/3551 [==============================] - ETA: 0s - loss: 2.4337 - acc: 0.1250
Epoch 00001: val_acc improved from -inf to 0.14973, saving model to model.h5
3551/3551 [==============================] - 7s 2ms/sample - loss: 2.4337 - acc: 0.1250 - val_loss: 2.4161 - val_acc: 0.1497 - lr: 1.0000e-04
Epoch 2/100
3530/3551 [============================>.] - ETA: 0s - loss: 2.3586 - acc: 0.1637
Epoch 00002: val_acc improved from 0.14973 to 0.20321, saving model to model.h5
3551/3551 [==============================] - 6s 2ms/sample - loss: 2.3581 - acc: 0.1639 - val_loss: 2.2595 - val_acc: 0.2032 - lr: 1.0000e-04
Epoch 3/100
3550/3551 [============================>.] - ETA: 0s - loss: 1.8931 - acc: 0.3561
Epoch 00003: val_acc improved from 0.20321 to 0.47059, saving model to model.h5
3551/3551 [==============================] - 6s 2ms/sample - loss: 1.8931 - acc: 0.3560 - val_loss: 1.5557 - val_acc: 0.4706 - lr: 1.0000e-04
Epoch 4/

In [0]:
)# get the metric names so we can use evaulate_generator
model.metrics_names

In [0]:
# Print the validation loss and accuracy.

# Here the best epoch will be used.
model.load_weights('model.h5')

val_loss, val_acc = \
model.evaluate_generator(test_gen, 
                        steps=len(df_val))

print('val_loss:', val_loss)
print('val_acc:', val_acc)

In [0]:
# make a prediction
predictions = model.predict_generator(test_gen, steps=len(df_val), verbose=1)

In [0]:
predictions.shape

In [0]:
# This is how to check what index keras has internally assigned to each class. 
test_gen.class_indices

In [0]:
# Put the predictions into a dataframe.
# The columns need to be ordered to match the output of the previous cell

class_dict = train_gen.class_indices

# Get a list of the dict keys.
cols = class_dict.keys()

df_preds = pd.DataFrame(predictions, columns=cols)

df_preds.head()

In [0]:
# Get the labels of the test images.

test_labels = test_gen.classes

In [0]:
# argmax returns the index of the max value in a row
cm = confusion_matrix(test_labels, predictions.argmax(axis=1))

In [0]:
# Define the labels of the class indices. These need to match the 
# order shown above.
cm_plot_labels = cols

plot_confusion_matrix(cm, cm_plot_labels, title='Confusion Matrix')